In [2]:
import pandas as pd
import plotly.express as px
import plotly.io as pio
pio.templates.default = "plotly_dark"

In [3]:
df = pd.read_csv('~/Projects/LEGO_Data/sets.csv')
df.columns = df.columns.str.lower()
df = df.drop(columns=['category','usd_msrp','packaging','num_instructions','theme_group','availability','minifigures','set_id'])
df = df.dropna()

Reading the csv file, changing all characters to lowerspace, deleting unnecessary columns which either have too many NANs or are useless for the research.

In [4]:
df['pieces'] = df.pieces.fillna(-1).astype(int)
df['subtheme'] = df.subtheme.fillna('-')
df['total_quantity'] = df.total_quantity.astype(int)
df['current_price'] = df.current_price.astype(int)
df['owned'] = df.owned.astype(int)
df = df[~df['rating'].isin([0])]
df = df.rename(columns={'total_quantity': 'last_sales', 'current_price': 'price'})
df = df.reset_index(drop=True)

Changing columns' types to appropriate ones.

In [5]:
df

,name,year,theme,subtheme,pieces,owned,rating,last_sales,price
0,Police Launch,1976,LEGOLAND,Boats,53,1118,3.2,1,149
1,Taxi,1979,Town,Classic,25,1618,3.1,1,99
2,Auto Service,1979,Town,Classic,43,1253,3.3,1,188
3,Trees and Flowers,1980,Town,Accessories,12,1842,3.8,1,49
4,Highway Repair,1980,Town,Maintenance,55,2166,3.9,1,222
...,...,...,...,...,...,...,...,...,...
3977,Pagani Utopia,2023,Speed Champions,Pagani,249,2530,4.5,1,25
3978,2 Fast 2 Furious Nissan Skyline GT-R (R34),2023,Speed Champions,Nissan,319,4755,4.4,43,14
3979,Fighter Plane Chase,2023,Indiana Jones,Last Crusade,387,1882,4.7,2,27
3980,Escape from the Lost Tomb,2023,Indiana Jones,Raiders of the Lost Ark,600,1721,4.5,4,33


In [6]:
df[['pieces', 'owned', 'price']].describe()

,pieces,owned,price
count,3982.000000,3982.000000,3982.000000
mean,354.856856,4589.500753,94.257911
std,653.214794,3697.342874,196.214028
min,1.000000,401.000000,0.000000
25%,59.000000,1971.500000,16.000000
50%,162.500000,3427.500000,39.000000
75%,391.000000,6106.250000,98.000000
max,11695.000000,29429.000000,5499.000000


Descriptive statistics for 'pieces', 'owned' and 'price' fields. 

In [7]:
price_per_year = df.groupby('year')['price'].mean().reset_index().set_index('year')
price_per_year['price'] = price_per_year.price.astype(int)
fig = px.bar(price_per_year, 
             x=price_per_year.index, 
             y='price', 
             color='price',
             title='Average retail price for lego sets',
             labels={'price': 'Current retail price',
                     'year': 'Year of release'})
fig.show()

Graph of the average cost of lego sets based on their release year.

In [8]:
popular = df.sort_values('owned', ascending=False).head(10)
popular

,name,year,theme,subtheme,pieces,owned,rating,last_sales,price
1621,Parisian Restaurant,2014,Creator Expert,Modular Buildings Collection,2469,29429,4.6,40,299
1630,Ghostbusters Ecto-1,2014,Ideas,Licensed,508,27851,4.4,29,114
3445,Mandalorian Battle Pack,2020,Star Wars,The Mandalorian,102,27799,4.1,434,19
1202,Volkswagen T1 Camper Van,2011,Advanced models,Vehicles,1334,26974,4.4,27,135
1409,Palace Cinema,2013,Creator Expert,Modular Buildings Collection,2196,26341,4.2,34,449
2404,Assembly Square,2017,Creator Expert,Modular Buildings Collection,4002,26028,4.4,20,299
1200,Pet Shop,2011,Advanced models,Modular Buildings Collection,2032,25758,4.3,47,325
1862,Detective's Office,2015,Creator Expert,Modular Buildings Collection,2262,24706,4.5,16,490
2111,Rey's Speeder,2015,Star Wars,The Force Awakens,193,24638,3.9,27,26
1029,Snowtrooper Battle Pack,2010,Star Wars,Episode V,74,24340,3.9,81,29


Ten most popular lego sets sorted by ownership

In [9]:
sales = df.groupby('year')[['price', 'last_sales']].mean().reset_index().set_index('year')
sales['last_sales'] = sales.last_sales.astype(int)
fig = px.scatter(sales, 
             x=sales.index, 
             y='last_sales', 
             size='price',
             opacity=0.9,
             title='Amount of sets sold in the last 6 months',
             labels={'year': 'Year of release',
                     'last_sales': 'Sales in the last 6 months'})
fig.show()

Amount of sets sold in the last 6 months based on the set's release, size is based on the price.

In [10]:
var = df.groupby('theme')['name'].count().sort_values(ascending=False).head(3)
var
popular_themes = df[df.theme.isin(var.index)]

In [11]:
import plotly.graph_objects as go


def make_hist(df, x, color, x_text, y_text, title_text):
    fig = go.Figure()
    for s_color in df[color].unique():
        fig.add_trace(go.Histogram(
            x=df[df[color] == s_color][x],
            histnorm='probability',
            nbinsx=30,
            name=s_color
        ))
    fig.update_layout(
        xaxis_title_text=x_text,
        yaxis_title_text=y_text,
        bargap=0.2,
        bargroupgap=0.1,
        title=title_text,
    )
    return fig

In [12]:
make_hist(popular_themes[popular_themes.price < popular_themes.price.quantile(.99)],
          'price', 
          'theme',
          'Price',
          'Probability',
          'Prices of sets from 3 most popular themes')

Probability means that if you take a random set from a specific theme (Star Wars, Creator, City), that set has a Probability percent of falling into a spefic price range on the graph.

In [13]:
# f = lambda x: (x[:3], x[-2:-1])
# y1, y2 = f(df.groupby('theme').price.mean().sort_values())
# p = df.groupby('theme').owned.sum()
# p[p.index.isin(y1.index) | p.index.isin(y2.index)]
counts = df.groupby('theme').subtheme.nunique().sort_values()
counts

theme
4 Juniors                    1
Hidden Side                  1
DC Super Hero Girls          1
LEGOLAND                     1
Minions: The Rise of Gru     1
                            ..
Bionicle                    36
City                        38
Collectable Minifigures     41
Friends                     41
Star Wars                   42
Name: subtheme, Length: 87, dtype: int64

In [14]:
def split_quantiles(row, counts):
    col = counts.index.name
    d = dict(counts)
    if d[row[col]] > counts.quantile(.75):
        return 4
    elif d[row[col]] > counts.quantile(.50):
        return 3
    elif d[row[col]] > counts.quantile(.25):
        return 2
    else:
        return 1

In [15]:
df['group'] = df.apply(lambda x: split_quantiles(x, counts), axis=1)
df

,name,year,theme,subtheme,pieces,owned,rating,last_sales,price,group
0,Police Launch,1976,LEGOLAND,Boats,53,1118,3.2,1,149,1
1,Taxi,1979,Town,Classic,25,1618,3.1,1,99,4
2,Auto Service,1979,Town,Classic,43,1253,3.3,1,188,4
3,Trees and Flowers,1980,Town,Accessories,12,1842,3.8,1,49,4
4,Highway Repair,1980,Town,Maintenance,55,2166,3.9,1,222,4
...,...,...,...,...,...,...,...,...,...,...
3977,Pagani Utopia,2023,Speed Champions,Pagani,249,2530,4.5,1,25,4
3978,2 Fast 2 Furious Nissan Skyline GT-R (R34),2023,Speed Champions,Nissan,319,4755,4.4,43,14,4
3979,Fighter Plane Chase,2023,Indiana Jones,Last Crusade,387,1882,4.7,2,27,3
3980,Escape from the Lost Tomb,2023,Indiana Jones,Raiders of the Lost Ark,600,1721,4.5,4,33,3


Added a new column 'group' which assigns a value to a set from 1 to 4, based on the amount of subthemes a set's theme has (4 - has more than 30 subthemes, 1 - has less than 10).

In [20]:
group_4 = df.drop(df[df['group'] != 4].index).groupby('year')['price'].mean().reset_index().set_index('year')
group_4['group_4'] = group_4.price.astype(int)
group_4 = group_4.drop(columns={'price'})
group_3 = df.drop(df[df['group'] != 3].index).groupby('year')['price'].mean().reset_index().set_index('year')
group_3['group_3'] = group_3.price.astype(int)
group_3 = group_3.drop(columns={'price'})
group_2 = df.drop(df[df['group'] != 2].index).groupby('year')['price'].mean().reset_index().set_index('year')
group_2['group_2'] = group_2.price.astype(int)
group_2 = group_2.drop(columns={'price'})
group_1 = df.drop(df[df['group'] != 1].index).groupby('year')['price'].mean().reset_index().set_index('year')
group_1['group_1'] = group_1.price.astype(int)
group_1 = group_1.drop(columns={'price'})
frame = [group_4, group_3, group_2, group_1]
so = pd.concat(frame)
px.line(so, 
        x=so.index, 
        y=['group_1','group_2','group_3','group_4'],
        title='Average price per group',
        labels={'value': 'Average price',
                'year': 'Year'})

A graph grouped by average price per year per group.

Hypothesis: was the amount of pieces increasing throughout the years and did it effect customers' satisfaction?

In [17]:
pieces_per_year = df.groupby('year')['pieces'].mean().reset_index().set_index('year')
pieces_per_year['pieces'] = pieces_per_year.pieces.astype(int)
fig = px.line(pieces_per_year, 
             x=pieces_per_year.index, 
             y='pieces',
             title='Average amount of pieces in a set by year',
             labels={'pieces': 'Pieces',
                     'year': 'Year'})
fig.show()

Apparently, the average amount of pieces in a set has been fluctuatiating, but increased in the long run.

In [18]:
rating_per_year = df.groupby('year')['rating'].mean().reset_index().set_index('year')
rating_per_year['rating'] = rating_per_year.rating.astype(float)
fig = px.line(rating_per_year, 
             x=rating_per_year.index, 
             y='rating',
             title='Rating per year',
             labels={'rating': 'Rating',
                     'year': 'Year'})
fig.show()

Seems like the amount of pieces didn't effect the satisfaction of customers, as from 2000 to present time the average rating of a set has been fluctuation, while the average amount of pieces increased.